In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score

# append the filepath to where torch is installed
sys.path.append('/home/millerm/.local/lib/python3.10/site-packages')
# sys.path.append('/home/username/.local/lib/python3.10/site-packages')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Load and Preprocess Data

In [2]:
file_name = "ml4h_data/project2/project2_TS_input/ptbdb_train.csv"
df_train = pd.read_csv(file_name,header=None)
x_train = df_train.iloc[:, df_train.columns != 187]
x_train = x_train.values.reshape(-1, 1, 187)
train_target = df_train.iloc[:, 187]
train_target = train_target.values
train_target

array([1., 1., 0., ..., 1., 1., 1.])

In [3]:
file_name = "ml4h_data/project2/project2_TS_input/ptbdb_test.csv"
df_test = pd.read_csv(file_name,header=None)
x_test = df_test.iloc[:, df_test.columns != 187]
x_test = x_test.values.reshape(-1, 1, 187)
test_target = df_test.iloc[:, 187]
test_target = test_target.values
test_target

array([0., 1., 0., ..., 1., 1., 0.])

In [4]:
from setsloaders import create_datasets, create_loaders

In [5]:
device = torch.device('cuda')
device

device(type='cuda')

In [6]:
datasets = create_datasets(x_train, x_test, train_target, test_target, seed=123)
trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

## Vanilla CNN

In [7]:
input_channels = 1
num_classes = 2
output_size = 2
train_loader = trn_dl
test_loader = tst_dl
num_epochs = 30
learning_rate = 0.005

In [8]:
class cnn(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3)
        self.fc1 = nn.Linear(5376, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = cnn(input_channels, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss}")
        
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))

Epoch: 0, Loss: 0.4388217628002167
Epoch: 1, Loss: 0.39442265033721924
Epoch: 2, Loss: 0.33628031611442566
Epoch: 3, Loss: 0.23043236136436462
Epoch: 4, Loss: 0.15206879377365112
Epoch: 5, Loss: 0.10935600847005844
Epoch: 6, Loss: 0.11535815894603729
Epoch: 7, Loss: 0.11228644102811813
Epoch: 8, Loss: 0.04675921052694321
Epoch: 9, Loss: 0.089759960770607
Epoch: 10, Loss: 0.0822092592716217
Epoch: 11, Loss: 0.05096311494708061
Epoch: 12, Loss: 0.06558927893638611
Epoch: 13, Loss: 0.03052239678800106
Epoch: 14, Loss: 0.02178395353257656
Epoch: 15, Loss: 0.03614456206560135
Epoch: 16, Loss: 0.007159082219004631
Epoch: 17, Loss: 0.04260924085974693
Epoch: 18, Loss: 0.01644989103078842
Epoch: 19, Loss: 0.018730537965893745
Epoch: 20, Loss: 0.024351218715310097
Epoch: 21, Loss: 0.04006785899400711
Epoch: 22, Loss: 0.026187248528003693
Epoch: 23, Loss: 0.006945765111595392
Epoch: 24, Loss: 0.013442585244774818
Epoch: 25, Loss: 0.003205548506230116
Epoch: 26, Loss: 0.003156254766508937
Epoch: 

In [9]:
model.eval()
predictions = []
ground_truth = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())

predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

precision = precision_score(ground_truth, predictions, average='binary')  # assuming binary classification
recall = recall_score(ground_truth, predictions, average='binary')  # assuming binary classification
f1 = (2*precision*recall)/(precision+recall)

print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1-Score: {:.2f}'.format(f1))

Precision: 0.99
Recall: 0.99
F1-Score: 0.99


## Residual Blocks

In [10]:
input_channels = 1
num_classes = 2
train_loader = trn_dl
test_loader = tst_dl
num_epochs = 30
learning_rate = 0.005

In [11]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out += residual  # Residual connection
        out = self.relu(out)
        return out

class rescnn(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(rescnn, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3)
        self.residual_block1 = ResidualBlock(64, 64, kernel_size=3)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.residual_block2 = ResidualBlock(128, 128, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3)
        self.residual_block3 = ResidualBlock(256, 256, kernel_size=3)
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(5376, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.residual_block1(x)
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.residual_block2(x)
        x = self.pool2(x)
        x = self.relu(self.conv3(x))
        x = self.residual_block3(x)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
model = rescnn(input_channels, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss}")
        
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))

Epoch: 0, Loss: 0.3979777991771698
Epoch: 1, Loss: 0.38194504380226135
Epoch: 2, Loss: 0.40125879645347595
Epoch: 3, Loss: 0.19134031236171722
Epoch: 4, Loss: 0.1398007720708847
Epoch: 5, Loss: 0.108409583568573
Epoch: 6, Loss: 0.0878017470240593
Epoch: 7, Loss: 0.15512025356292725
Epoch: 8, Loss: 0.06042991578578949
Epoch: 9, Loss: 0.10401635617017746
Epoch: 10, Loss: 0.04419603571295738
Epoch: 11, Loss: 0.04062766954302788
Epoch: 12, Loss: 0.05520084127783775
Epoch: 13, Loss: 0.10112343728542328
Epoch: 14, Loss: 0.04387304186820984
Epoch: 15, Loss: 0.05715956911444664
Epoch: 16, Loss: 0.12227906286716461
Epoch: 17, Loss: 0.019189991056919098
Epoch: 18, Loss: 0.019750172272324562
Epoch: 19, Loss: 0.019334880635142326
Epoch: 20, Loss: 0.022977938875555992
Epoch: 21, Loss: 0.006403555162250996
Epoch: 22, Loss: 0.00873741414397955
Epoch: 23, Loss: 0.03909185528755188
Epoch: 24, Loss: 0.002602543216198683
Epoch: 25, Loss: 0.043907877057790756
Epoch: 26, Loss: 0.0070802513509988785
Epoch: 

In [13]:
model.eval()
predictions = []
ground_truth = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())

predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

precision = precision_score(ground_truth, predictions, average='binary')  # assuming binary classification
recall = recall_score(ground_truth, predictions, average='binary')  # assuming binary classification
f1 = (2*precision*recall)/(precision+recall)

print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1-Score: {:.2f}'.format(f1))

Precision: 0.99
Recall: 0.99
F1-Score: 0.99
